In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Activation, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import TensorBoard

from dataset import getDataset
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import os
import cv2
import time
import random

In [2]:
IMG_SIZE = 224
X, y = getDataset(IMG_SIZE, normalize=True)

print(X.shape, y.shape)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


X_aug, y_aug = [], []

cnt = 0
for _X, _y in datagen.flow(X, y):    
    for i in range(_X.shape[0]):
        cnt += 1
        X_aug.append(_X[i])
        y_aug.append(_y[i])
    
    if cnt >= 1000:
        break

X = np.array(X_aug).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y_aug).reshape(-1, 1)

print(X.shape, y.shape)

(92, 224, 224, 3) (92, 1)


/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(1012, 224, 224, 3) (1012, 1)


In [3]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 222, 222, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0

In [6]:
history = model.fit(X, y, validation_split=0.2, batch_size=32, epochs=35, shuffle=True)

Epoch 1/35
26/26 [==============================] - 24s 904ms/step - loss: 0.6651 - accuracy: 0.6489 - val_loss: 0.6294 - val_accuracy: 0.6650
Epoch 2/35
26/26 [==============================] - 17s 660ms/step - loss: 0.6005 - accuracy: 0.6675 - val_loss: 0.5096 - val_accuracy: 0.6650
Epoch 3/35
26/26 [==============================] - 17s 662ms/step - loss: 0.4847 - accuracy: 0.7750 - val_loss: 0.3859 - val_accuracy: 0.8374
Epoch 4/35
26/26 [==============================] - 17s 662ms/step - loss: 0.4317 - accuracy: 0.8096 - val_loss: 0.3605 - val_accuracy: 0.8374
Epoch 5/35
26/26 [==============================] - 17s 662ms/step - loss: 0.3926 - accuracy: 0.8418 - val_loss: 0.3997 - val_accuracy: 0.8424
Epoch 6/35
26/26 [==============================] - 17s 663ms/step - loss: 0.3335 - accuracy: 0.8727 - val_loss: 0.2988 - val_accuracy: 0.8670
Epoch 7/35
26/26 [==============================] - 17s 660ms/step - loss: 0.2980 - accuracy: 0.8616 - val_loss: 0.3020 - val_accuracy: 0.8621